In [1]:
## Load necessary libraries
%matplotlib inline
from constructIDF import *
import pandas as pd
import numpy as np
import itertools
import argparse
import matplotlib.pyplot as plt

from matplotlib import rcParams

rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

### Step 1: Read file with daily rainfall records from MACA.

The data used in this example was downloaded from the MACA portal. The data corresponds to downscaled daily rainfall observations from 10 different GCM extracted from the closest grid cell center to Ann Arbor lat, lon coordinates (Latitude:42.2981, Longitude:-83.6639)

In [9]:
"[*]"

# Specify path to hourly rainfall time series

## REMEMBER to specify the path to where the historical data from MACA was stored.

historical_path = ""

In [10]:
## Reading data in
## Historical_data is the historical MACA data for Ann Arbor.

historical_data = pd.read_csv(historical_path, skiprows=26, parse_dates=["yyyy-mm-dd"])

### Step 2: Construct Annual Maximum Series

We only have daily observations, so we can only do starting
from 24. We will only create 24-hour IDF curves in this tutorial.

In [11]:
## Reformatting data so that it is in the format 
## required by the methods we used in HW1

historical_data['year'] = historical_data["yyyy-mm-dd"].dt.year

#The following line of code gets the maximum daily rainfall 
# on each year
historicalAMS = historical_data.groupby(pd.Grouper(key="yyyy-mm-dd", freq='A')).max()


cols = list(historicalAMS)
cols.insert(0, cols.pop(cols.index('year')))
historicalAMS = historicalAMS.loc[:, cols]
out = historicalAMS.reset_index(drop=True)

In [12]:
#Take a look

#########################
#  Historical Model     #
#     AMS TABLE         #
#                       #
#########################

out

,year,pr_bcc-csm1-1_historical(mm),pr_bcc-csm1-1-m_historical(mm),pr_BNU-ESM_historical(mm),pr_CanESM2_historical(mm),pr_CCSM4_historical(mm),pr_CNRM-CM5_historical(mm),pr_CSIRO-Mk3-6-0_historical(mm),pr_GFDL-ESM2M_historical(mm),pr_GFDL-ESM2G_historical(mm),pr_HadGEM2-CC365_historical(mm)
0,1950,32.170700,28.330273,25.961569,41.700222,23.778496,40.512627,30.651091,34.664284,40.512627,40.512627
1,1951,38.417217,30.189684,28.345760,51.715496,38.092495,29.494617,34.913235,46.009525,38.343445,47.898281
2,1952,40.362488,36.423500,53.989502,47.898281,40.362488,39.591415,24.854980,46.715599,42.725613,33.969921
3,1953,35.092014,34.664284,52.026752,41.975277,38.320995,30.128036,27.875061,35.953228,27.804882,36.172832
4,1954,44.760929,51.715496,42.725613,32.958679,40.126492,45.494953,39.536640,38.991837,58.161530,38.063091
5,1955,46.608860,28.903296,47.898281,41.021931,35.660465,35.062305,40.512627,52.803215,40.023609,65.595009
6,1956,46.715599,65.595009,36.172832,28.345760,35.062305,38.320995,34.205025,32.816669,41.021931,27.459637
7,1957,32.048145,41.975277,44.760929,47.190266,36.778805,35.092014,26.854403,38.343445,45.494953,37.490189
8,1958,65.595009,52.026752,31.787653,34.985138,26.592812,36.684879,42.941067,32.277603,28.192711,41.700222
9,1959,38.063091,46.715599,39.570255,31.305250,30.741625,52.026752,33.897297,42.725613,30.758911,25.397488


### Step 3: Fit Generalized Extreme Value and obtain rainfall depths

The next step is to fit a generalized extreme value distribution to each duration's AMS. Once the parameters (location, scale and shape) are estimated, these are used to retrieve the return levels (in this case, rainfall depth) for different quantiles feed into the inverse of the CDF. Usually, the quantiles are equal to the inverse of the average recurrence interval (ARI) (e.g. 1/2 = 2-year).

`constructIDF` has one method that merges all these steps, but we need to specify if we want to construct confidence intervals. The method implemented in `constructIDF` is bootstrapping, so we also need to specify the number of bootsrapped samples. Default value is 1000, and using a smaller number is not recommended.

Other specification is the confidence level, alpha, used to estimate the confidence intervals. Default is 0.9 (90% confidence interval).


In this tutorial, we will compute confidence intervals at a 90% confidence level using 1000 bootstrapped samples.

```python
ci = True
alpha = 0.9
number_bootstrap = 1000
```

In [13]:
# Specifying values #

ci = True
alpha = 0.9
number_bootstrap = 100

In [14]:
# Feeding the data and our specifications to the method.

data = IDF(out, ci, number_bootstrap, alpha)

In [15]:
# Construct IDF from the data we feed above and our specifications.
# Some errors will be displayed, no worries. This will take long time because
# of the number of bootsrapped samples.
# The constructed IDF is by default for the following ARI:
# 2-, 5-, 10-, 25-, 50-, 100-, 200-year

data.construct_IDF()


In [16]:
######################################
#                                    # 
#          Historical Model          #
#             IDF TABLE              #
#                                    #
######################################

## We can access the dataframe with confidence bounds:

## Note that we no longer have "DURATIONS" by column. 
## We only did for one duration since we only have daily data.
## Now, each column corresponds to the 24-hour historical IDF
## curve for each model.

data.idf

,pr_bcc-csm1-1_historical(mm),pr_bcc-csm1-1-m_historical(mm),pr_BNU-ESM_historical(mm),pr_CanESM2_historical(mm),pr_CCSM4_historical(mm),pr_CNRM-CM5_historical(mm),pr_CSIRO-Mk3-6-0_historical(mm),pr_GFDL-ESM2M_historical(mm),pr_GFDL-ESM2G_historical(mm),pr_HadGEM2-CC365_historical(mm)
L2-yr,37.183863,36.986718,36.999564,35.850703,35.778862,35.997130,34.813536,36.976174,36.568260,36.589381
L5-yr,44.608210,44.744893,44.187985,43.395488,43.929594,43.827752,43.481172,44.482655,44.361217,43.629640
L10-yr,49.230980,48.922676,49.143430,48.455171,49.032275,48.329864,49.701674,49.174785,49.484268,48.196161
L25-yr,53.504905,53.942331,54.681520,53.610092,55.052929,53.399259,57.267989,55.192823,55.058213,53.795651
L50-yr,56.742762,57.294396,57.583051,56.838379,58.016182,55.988801,60.244170,59.157641,59.374037,57.637776
L100-yr,59.774200,59.971962,59.950641,60.369734,60.814038,58.116156,63.863830,62.001471,62.278027,60.593471
L200-yr,62.620192,62.104683,63.039450,63.152854,63.546717,60.108194,67.505084,64.888432,65.456911,63.225614
2-yr,39.029780,38.532302,39.430258,38.252933,38.236991,38.316291,37.072180,38.972665,38.409021,38.514957
5-yr,46.960978,47.294543,47.507294,46.917274,47.158230,46.708220,46.593613,47.110001,46.971959,46.744190
10-yr,52.328673,52.764548,52.705441,52.757862,52.694275,51.913365,53.438609,52.715538,52.485173,52.274170


### Step 3: Obtain IDF values from future data

Now, we need to repeat the same process but inputing the
future model data.

In [17]:
"[*]"

"""
Specify here the path to the data from RCP 8.5 or RCP 4.5 downloaded from MACA
"""
future_path = ""

In [18]:
## Reading data in
## Historical_data is the historical MACA data for Ann Arbor.

future_data = pd.read_csv(future_path, skiprows=26, parse_dates=["yyyy-mm-dd"])

In [19]:
## Reformatting data so that it is in the format 
## required by the methods we used in HW1

future_data['year'] = future_data["yyyy-mm-dd"].dt.year

#The following line of code gets the maximum daily rainfall 
# on each year
futureAMS = future_data.groupby(pd.Grouper(key="yyyy-mm-dd", freq='A')).max()


cols = list(futureAMS)
cols.insert(0, cols.pop(cols.index('year')))
futureAMS = futureAMS.loc[:, cols]
future_out = futureAMS.reset_index(drop=True)

In [20]:
#Take a look

#########################
#      Future Model     #
#       AMS TABLE       #
#                       #
#########################

future_out

,year,pr_bcc-csm1-1_rcp85(mm),pr_bcc-csm1-1-m_rcp85(mm),pr_BNU-ESM_rcp85(mm),pr_CanESM2_rcp85(mm),pr_CCSM4_rcp85(mm),pr_CNRM-CM5_rcp85(mm),pr_CSIRO-Mk3-6-0_rcp85(mm),pr_GFDL-ESM2M_rcp85(mm),pr_GFDL-ESM2G_rcp85(mm),pr_HadGEM2-CC365_rcp85(mm)
0,2006,41.762489,65.705200,41.413303,44.896496,40.287437,49.113220,32.486240,49.317863,39.887020,66.769920
1,2007,38.984734,28.114281,31.947811,46.017780,29.723171,33.605335,54.532627,43.626713,77.920204,45.774925
2,2008,45.069363,34.916054,36.909401,42.551731,62.972191,42.009243,62.468037,31.248665,52.733070,35.564102
3,2009,48.521442,42.810760,33.440716,79.665039,27.621651,42.226654,39.272945,41.084148,32.946880,33.020657
4,2010,56.192223,34.005508,30.876858,47.956028,40.311985,38.033913,57.396362,29.048100,35.026276,25.479580
5,2011,32.948307,78.111465,30.275681,53.245522,32.609035,50.956348,33.942802,38.154999,42.439518,41.509102
6,2012,44.667282,55.735611,48.518406,33.023602,47.354401,42.440079,106.742500,35.227810,60.910507,25.867962
7,2013,36.793396,34.378925,42.378456,22.895462,31.897778,39.370762,50.096195,53.535370,66.857185,47.343544
8,2014,37.680214,52.181297,37.767151,43.155308,30.192486,37.534046,36.844730,47.618202,39.964985,50.403439
9,2015,36.773064,51.475437,80.702110,35.786327,42.850693,43.854259,95.317520,38.230331,40.249603,41.111614


In [21]:
# Feeding the data and our specifications to the method,
# same as when we calculated the historical model IDF values.

future_data = IDF(future_out, ci, number_bootstrap, alpha)

In [22]:
# Construct IDF from the data we feed above and our specifications.
# Some errors will be displayed, no worries. This will take long time because
# of the number of bootsrapped samples.
# The constructed IDF is by default for the following ARI:
# 2-, 5-, 10-, 25-, 50-, 100-, 200-year

future_data.construct_IDF()


In [23]:
######################################
#                                    # 
#          Future Model              #
#           IDF TABLE                #
#                                    #
######################################

## We can access the dataframe with confidence bounds:

## Note that we no longer have "DURATIONS" by column. 
## We only did for one duration since we only have daily data.
## Now, each column corresponds to the 24-hour historical IDF
## curve for each model.

future_data.idf

,pr_bcc-csm1-1_rcp85(mm),pr_bcc-csm1-1-m_rcp85(mm),pr_BNU-ESM_rcp85(mm),pr_CanESM2_rcp85(mm),pr_CCSM4_rcp85(mm),pr_CNRM-CM5_rcp85(mm),pr_CSIRO-Mk3-6-0_rcp85(mm),pr_GFDL-ESM2M_rcp85(mm),pr_GFDL-ESM2G_rcp85(mm),pr_HadGEM2-CC365_rcp85(mm)
L2-yr,44.483248,42.565912,39.359303,40.939328,39.811141,41.360535,46.480155,43.526908,44.569631,44.068842
L5-yr,56.699601,54.562439,47.779962,55.551058,50.125671,51.214415,60.443790,54.609784,57.403131,56.699212
L10-yr,64.599171,62.742768,53.745950,67.587706,56.426054,57.291553,70.499109,61.662946,65.429175,65.090965
L25-yr,74.928261,74.124374,60.872025,86.935549,64.043329,64.350171,82.838650,70.427667,74.806888,76.060684
L50-yr,81.903956,82.605758,65.598516,103.692481,68.914076,69.386320,91.654168,77.262744,82.831706,84.591416
L100-yr,89.383013,91.049633,70.002035,122.863220,74.173589,73.894371,99.583205,83.938011,87.552026,93.353841
L200-yr,96.169105,99.538123,74.927221,145.152409,78.487174,78.390293,106.964639,91.139476,93.687421,101.295030
2-yr,46.554041,44.646575,41.056699,43.715795,41.836636,43.191865,49.252461,45.459608,47.047046,46.139996
5-yr,60.023059,58.272490,50.649405,61.463784,52.645753,53.901522,64.353265,57.440909,60.932869,60.013287
10-yr,69.891973,68.107531,57.460282,77.630795,59.956535,61.141819,75.568613,66.413852,70.987810,70.190768


In [24]:
## Find change factors by finding the ratio between
## historical IDF model values and future IDF model values.

## Find change factor

change_factors = pd.DataFrame(future_data.idf.values/data.idf.values)

In [25]:
## Make table look pretty and understandable 
change_factors.columns = [x.rstrip("_rcp85(mm)") for x in future_data.idf.columns]
change_factors['return_period'] = future_data.idf.index
change_factors.set_index('return_period', inplace=True)

In [26]:
####################################
#                                  # 
#          CHANGE FACTORS          #
#             TABLE                #
#                                  #
####################################

change_factors

,pr_bcc-csm1-1,pr_bcc-csm1-1-,pr_BNU-ESM,pr_CanESM2,pr_CCSM4,pr_CNRM-CM,pr_CSIRO-Mk3-6-0,pr_GFDL-ESM2M,pr_GFDL-ESM2G,pr_HadGEM2-CC36
return_period,,,,,,,,,,
L2-yr,1.196305,1.150843,1.063777,1.141939,1.112700,1.148995,1.335117,1.177161,1.218806,1.204416
L5-yr,1.271058,1.219412,1.081289,1.280111,1.141046,1.168538,1.390114,1.227665,1.293994,1.299557
L10-yr,1.312165,1.282488,1.093655,1.394850,1.150794,1.185428,1.418445,1.253955,1.322222,1.350543
L25-yr,1.400400,1.374141,1.113210,1.621627,1.163305,1.205076,1.446509,1.276029,1.358687,1.413882
L50-yr,1.443426,1.441777,1.139198,1.824339,1.187842,1.239289,1.521378,1.306048,1.395083,1.467638
L100-yr,1.495344,1.518203,1.167661,2.035179,1.219679,1.271494,1.559305,1.353807,1.405825,1.540658
L200-yr,1.535752,1.602747,1.188577,2.298430,1.235110,1.304153,1.584542,1.404557,1.431284,1.602120
2-yr,1.192783,1.158679,1.041249,1.142809,1.094140,1.127245,1.328556,1.166449,1.224896,1.197976
5-yr,1.278148,1.232119,1.066140,1.310046,1.116364,1.154005,1.381161,1.219293,1.297218,1.283866


We can construct the future IDF at Ann Arbor by updating the historical curve (from Homework 1 Part I) with the climate signal that we estimated from the downscaled projections. 
Note that there are two options here:

1. Assume equal change in all storm durations to the estimated change in the 24-h rainfall depth (the change factors we computed above). 
2. Update the 24-hour only.

Option 1 entails a strong assumption, and there is evidence that shorter duration extremes will change substantially more than longer duration extremes You can read this paper here: 

    Prein, A. F., Rasmussen, R. M., Ikeda, K., Liu, C., Clark, M. P., & Holland, G. J. (2017). The future intensification of hourly precipitation extremes. Nature Climate Change, 7(1), 48–52. https://doi.org/10.1038/nclimate3168
    

Therefore, in this homework we will go by Option 2.


In [27]:
"[*]"

# Loading idf values from historical 24-hour duration IDF curve that you generated 
#using the tutorial Historical_IDF_Curves:

# Note that there is no "/" at the end of path_canvas

path_24h_Historical = "" # specify here

name_file = "24H_Pittsburgh.csv"

In [28]:
observed_idf = pd.read_csv("{}/{}".format(path_24h_Historical, name_file), index_col=0)

In [ ]:
observed_idf

In [30]:
######################################
#                                    # 
#            Future                  #
#           IDF TABLE                #
#                                    #
######################################

"""
This line of code is multiplying the historical IDF curve values
with the CHANGE FACTORS TABLE computed in the cells above
"""


updated_idf = change_factors.multiply(observed_idf["24H"], axis="index")

In [ ]:
## Take a look

updated_idf

### Step 4: Generate FUTURE IDF curves


#### For this homework, you will have to perform some statistics, you can directly do them here if you are a python-user. Otherwise, you need to save the data above as a .csv and open in Excel. To  save the data above uncomment the following cell and specify the folder where to save in the save_path variable.

Example: save_path = "\Users\tanialopez\Downloads\"

We can call the `plot_IDF` method to create the IDF curves and plot them.
We need to pass the path where the original data was stored, a path where to store
the figure and its format.

In [9]:
"[*]"

save_path = ""

In [ ]:
# This part generates a plot and saves in your folder specified above

# Hard coded params
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
idf_transposed = updated_idf.transpose()
dfmean = idf_transposed.drop([x for x in idf_transposed.columns if (
                x[:1] == 'L' or x[:1] == 'U')], axis=1)

dfmean = dfmean.transpose()
fig, axs = plt.subplots(figsize=(13, 10))
a1 = dfmean.plot(ax=axs)
fill_alpha = 0.3

legend = plt.legend(title='GCM Model', fontsize=13)
plt.setp(legend.get_title(), fontsize=15)
plt.ylabel('Precipitation Depth (mm)', {'fontsize': 18})
plt.xlabel('Return Period', {'fontsize': 18})
plt.title('Future (2006-2099) 24-hour IDF curves', fontsize=22)
plt.grid()

plt.savefig("{}/Figure.{}".format(save_path,
                                          'png'), bbox_inches='tight')